In [1]:
!pip install transformers

     |████████████████████████████████| 778kB 7.2MB/s 
     |████████████████████████████████| 1.1MB 34.6MB/s 
     |████████████████████████████████| 3.0MB 40.3MB/s 
     |████████████████████████████████| 890kB 28.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a7d22c2c7e703f47a59cdd342284c2bd8d0d661ecce1e6b3585eae8c855b3296
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import torch
import json
import csv
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np
import pandas as pd

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

print(f'Using device: {device}')

Using device: cuda


# Load GPT-2 Medium 

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', model_max_length=1024)
model = GPT2LMHeadModel.from_pretrained('gpt2')
model = model.to(device)

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Helper functions

In [ ]:
def generate_text(model, tokenizer, prompt, keywords, category):
  pass

# Reviews dataset

In [10]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
DATA_PATH = '/content/drive/My Drive/Data Science/Datasets'
!ls '$DATA_PATH'

reviews_nlp_encoded.txt


In [12]:
from torch.utils.data import Dataset, DataLoader

class ReviewsDataset(Dataset):
    def __init__(self, filename):
        super().__init__()

        with open(filename) as data_file:
          self.reviews = data_file.readlines()
        
    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        return self.reviews[idx].replace('\n', '')

review_dataset = ReviewsDataset(f'{DATA_PATH}/reviews_nlp_encoded.txt')
review_loader = DataLoader(review_dataset, batch_size=1, shuffle=True)

# Hyperparameters

In [13]:
BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 1e-5
WARMUP_STEPS = 5000
MAX_SEQ_LEN = 1000
from transformers import AdamW, get_linear_schedule_with_warmup

In [54]:
[1, 2, 3, 4][-2:]

[3, 4]

In [34]:
loss_history = torch.tensor([[1, 2, 3]], dtype=torch.float32)
torch.cat((loss_history, torch.tensor([[2, 3, 4]])), dim=0)

tensor([[1., 2., 3.],
        [2., 3., 4.]])

# Training

In [56]:
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=-1)
proc_seq_count = 0

loss_history = []
batch_count = 0
loss_every = 2

tmp_reviews_tens = None
models_folder = "trained_models"
if not os.path.exists(models_folder):
    os.mkdir(models_folder)

for epoch in range(EPOCHS):
    print(f"EPOCH {epoch+1}")
    epoch_loss_history = []
  
    for review in review_loader:
        # fit as many review sequences into MAX_SEQ_LEN sequence as possible
        review_tens = torch.tensor(tokenizer.encode(review[0], max_length=MAX_SEQ_LEN, truncation=True)).unsqueeze(0).to(device)
        
        work_reviews_tens = review_tens

        # the first review sequence in the sequence
        if not torch.is_tensor(tmp_reviews_tens):
            tmp_reviews_tens = review_tens
            continue
        else:
            # the next review does not fit in so we process the sequence and leave the last review 
            # as the start for next sequence 
            if tmp_reviews_tens.size()[1] + review_tens.size()[1] < MAX_SEQ_LEN:
                # add the review to sequence, continue and try to add more
                tmp_reviews_tens = torch.cat([tmp_reviews_tens, review_tens[:,1:]], dim=1)
                continue
            else:
                work_reviews_tens = tmp_reviews_tens
                tmp_reviews_tens = review_tens

        # sequence ready, process it trough the model
        outputs = model(work_reviews_tens, labels=work_reviews_tens)
        loss, logits = outputs[:2]                        
        loss.backward()
        epoch_loss_history.append(loss.detach().item())
                       
        proc_seq_count += 1
        if proc_seq_count == BATCH_SIZE:
            proc_seq_count = 0
            batch_count += 1
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()

        if batch_count == loss_every:
            batch_count = 0
            avg_loss = np.array(epoch_loss_history)[-loss_every*BATCH_SIZE+1:].mean()
            print(f'Avg. loss: {avg_loss:.4f}')
    
    loss_history.append(torch.tensor(epoch_loss_history))
    # store the model after each epoch to compare the performance of them
    torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_small_review_{epoch}.pt"))

EPOCH 1
Avg. loss: 3.2195
Avg. loss: 3.1996
Avg. loss: 3.1690
Avg. loss: 3.1972
Avg. loss: 3.2277
Avg. loss: 3.2295


KeyboardInterrupt: ignored

In [ ]:
# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
input = tokenizer.encode("<|startoftext|>~`bad~^sex~@", return_tensors='pt').to(device)
sample_outputs = model.generate(
    input,
    do_sample=True, 
    max_length=600, 
    top_k=30, 
    top_p=0.96, 
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Output:
----------------------------------------------------------------------------------------------------
0: <|startoftext|>~`bad~^sex~@This book is about the end of a series of books that started with "Toward a Perfect Life". It is, in this way, a bit of a book that should be read by a child. It is quite a bit of a novel, especially in the sense that it starts with a child's initial story and that he wants to follow, with a few things. It is one of those stories which you can really just enjoy, even at the end of each chapter. I am not sure if it was enough. However I enjoyed this book so much that I could still see it coming.
1: <|startoftext|>~`bad~^sex~@It was so annoying when I had to finish it after 4 straight minutes. Not sure what I should go on here. It's not worth the extra read. I will probably read a book now.
2: <|startoftext|>~`bad~^sex~@I was really excited to finally read this book. I loved the idea of the ending to the story but the fact that I am not sure what will